# GPU speed comparison

In [2]:
import torch

In [3]:
print(torch.cuda.is_available())        # True expected
print(torch.cuda.get_device_name(0))    # Quadro RTX 4000 expected

True
Quadro RTX 4000


In [4]:
import sys
import os


cwd = os.getcwd()
project_root = None
    
# Check if any of the specified directory names are in the current path
for dir_name in ("ML_competitions", "workspace"):
    if dir_name in cwd:
        project_root = cwd[:cwd.index(dir_name) + len(dir_name)]
        break
    
if project_root:
    if project_root not in sys.path:
        sys.path.append(project_root)
        print(f"Added to sys.path: {project_root}")
else:
    raise RuntimeError(f"None of {root_dir_names} found in current path: {cwd}")

In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

from my_library.models.custom_tabn import CustomTabNetModel
from my_library.configs.model_configs.tabn_configs import TabNetConfig
from my_library.configs.model_configs.fit_configs import FitConfig
from my_library.validations.validation_runner import ValidationRunner
from my_library.parameter_tunings.optuna_tuner import OptunaValidationTuner
from my_library.utils.timeit import timeit
from my_library.splitters.cross_validation import CrossValidationSplitter

from sklearn.metrics import roc_auc_score

In [6]:
# 擬似巨大データの生成
X, y = make_classification(n_samples=100_000, n_features=200, n_informative=50, n_redundant=50, random_state=42)
X = pd.DataFrame(X, columns=[f"feat_{i}" for i in range(X.shape[1])])
y = pd.Series(y, name="target")

In [7]:
X.shape, y.shape

((100000, 200), (100000,))

In [8]:
# 共通設定
feats = X.columns.tolist()
fit_config = FitConfig(feats=feats, early_stopping_rounds=10, epochs=100, batch_size=1024)
folds = CrossValidationSplitter(n_splits=5, shuffle=True, random_state=42).split(X, y)

In [9]:
# 実行関数の定義
@timeit
def run_runner(use_gpu: bool, parallel_mode: str):
    config = TabNetConfig(
        task_type="classification",
        params=dict(n_d=8, n_a=8, n_steps=3, batch_size=1024, max_epochs=100),
        use_gpu=use_gpu
    )
    runner = ValidationRunner(
        model_class=CustomTabNetModel,
        model_configs=config,
        metric_fn=roc_auc_score,
        predict_proba=True,
        parallel_mode=parallel_mode
    )
    results = runner.run(folds, fit_config)
    return results

In [10]:
# 時間計測（3パターン）
# results_cpu_serial = run_runner(use_gpu=False, parallel_mode=False)
# results_cpu_parallel = run_runner(use_gpu=False, parallel_mode=True)
results_gpu_serial = run_runner(use_gpu=True, parallel_mode=False)

2025-05-08 13:33:23,945 - my_library.validations.validation_runner - INFO - Initialized ValidationRunner | model=CustomTabNetModel task=classification
2025-05-08 13:33:23,946 - my_library.validations.validation_runner - INFO - Validation run started.
2025-05-08 13:33:23,947 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 13:33:23,948 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 13:33:23,948 - my_library.validations.validation_runner - INFO - [Fold 1] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=True
2025-05-08 13:33:23,949 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-08 13:33:23,949 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200



Early stopping occurred at epoch 40 with best_epoch = 20 and best_valid_accuracy = 0.9916


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-08 13:35:28,232 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-08 13:35:28,234 - my_library.validations.validator - INFO - Training completed.
2025-05-08 13:35:28,568 - my_library.validations.validation_runner - INFO - [Fold 1] Score=0.9916
2025-05-08 13:35:28,570 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 13:35:28,570 - my_library.validations.validation_runner - INFO - [Fold 2] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=True
2025-05-08 13:35:28,571 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-08 13:35:28,571 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200



Early stopping occurred at epoch 33 with best_epoch = 13 and best_valid_accuracy = 0.99275


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-08 13:37:13,225 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-08 13:37:13,227 - my_library.validations.validator - INFO - Training completed.
2025-05-08 13:37:13,560 - my_library.validations.validation_runner - INFO - [Fold 2] Score=0.9928
2025-05-08 13:37:13,562 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 13:37:13,562 - my_library.validations.validation_runner - INFO - [Fold 3] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=True
2025-05-08 13:37:13,563 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-08 13:37:13,563 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200


...

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-08 13:40:33,705 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-08 13:40:33,706 - my_library.validations.validator - INFO - Training completed.
2025-05-08 13:40:34,032 - my_library.validations.validation_runner - INFO - [Fold 3] Score=0.9917
2025-05-08 13:40:34,033 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 13:40:34,034 - my_library.validations.validation_runner - INFO - [Fold 4] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=True
2025-05-08 13:40:34,034 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-08 13:40:34,035 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200



Early stopping occurred at epoch 41 with best_epoch = 21 and best_valid_accuracy = 0.9919


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-08 13:42:43,665 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-08 13:42:43,667 - my_library.validations.validator - INFO - Training completed.
2025-05-08 13:42:44,001 - my_library.validations.validation_runner - INFO - [Fold 4] Score=0.9919
2025-05-08 13:42:44,002 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 13:42:44,002 - my_library.validations.validation_runner - INFO - [Fold 5] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=True
2025-05-08 13:42:44,003 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-08 13:42:44,003 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200



Early stopping occurred at epoch 49 with best_epoch = 29 and best_valid_accuracy = 0.9919


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-08 13:45:15,027 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-08 13:45:15,029 - my_library.validations.validator - INFO - Training completed.
2025-05-08 13:45:15,359 - my_library.validations.validation_runner - INFO - [Fold 5] Score=0.9919
2025-05-08 13:45:15,360 - my_library.validations.validation_runner - INFO - Validation completed | mean=0.9920 std=0.0004


In [11]:
# OptunaによるGPUチューニング
@timeit
def run_gpu_optuna():
    config = TabNetConfig(
        task_type="classification",
        params={},  # 初期は空でもOK
        use_gpu=True
    )
    tuner = OptunaValidationTuner(
        model_class=CustomTabNetModel,
        model_configs=config,
        param_space={
            "n_d": [8, 16],
            "n_steps": [3, 5],
            "learning_rate": (0.01, 0.2),
        },
        folds=folds,
        scoring=roc_auc_score,
        predict_proba=True,
        n_trials=5,
        maximize=True,
        parallel_mode=False,
    )
    return tuner.tune(fit_config, return_runner_results=True)


In [12]:
best_params, best_score, best_result = run_gpu_optuna()

[I 2025-05-08 13:46:21,675] A new study created in memory with name: no-name-a2ae1fda-e04b-4bab-9e2d-182727bb528b
2025-05-08 13:46:21,678 - my_library.validations.validation_runner - INFO - Initialized ValidationRunner | model=CustomTabNetModel task=classification
2025-05-08 13:46:21,679 - my_library.validations.validation_runner - INFO - Validation run started.
2025-05-08 13:46:21,680 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 13:46:21,681 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 13:46:21,682 - my_library.validations.validation_runner - INFO - [Fold 1] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=True
2025-05-08 13:46:21,683 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-08 13:46:21,683 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200



Early stopping occurred at epoch 58 with best_epoch = 38 and best_valid_accuracy = 0.99155


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-08 13:50:32,772 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-08 13:50:32,773 - my_library.validations.validator - INFO - Training completed.
2025-05-08 13:50:33,227 - my_library.validations.validation_runner - INFO - [Fold 1] Score=0.9915
2025-05-08 13:50:33,228 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 13:50:33,229 - my_library.validations.validation_runner - INFO - [Fold 2] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=True
2025-05-08 13:50:33,229 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-08 13:50:33,230 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200



Early stopping occurred at epoch 47 with best_epoch = 27 and best_valid_accuracy = 0.99345


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-08 13:53:58,360 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-08 13:53:58,362 - my_library.validations.validator - INFO - Training completed.
2025-05-08 13:53:58,806 - my_library.validations.validation_runner - INFO - [Fold 2] Score=0.9934
2025-05-08 13:53:58,808 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 13:53:58,808 - my_library.validations.validation_runner - INFO - [Fold 3] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=True
2025-05-08 13:53:58,809 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-08 13:53:58,809 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200


...


Early stopping occurred at epoch 58 with best_epoch = 38 and best_valid_accuracy = 0.99155


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-08 14:08:47,902 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-08 14:08:47,903 - my_library.validations.validator - INFO - Training completed.
2025-05-08 14:08:48,359 - my_library.validations.validation_runner - INFO - [Fold 1] Score=0.9915
2025-05-08 14:08:48,361 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 14:08:48,361 - my_library.validations.validation_runner - INFO - [Fold 2] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=True
2025-05-08 14:08:48,362 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-08 14:08:48,362 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200



Early stopping occurred at epoch 47 with best_epoch = 27 and best_valid_accuracy = 0.99345


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-08 14:12:16,852 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-08 14:12:16,853 - my_library.validations.validator - INFO - Training completed.
2025-05-08 14:12:17,286 - my_library.validations.validation_runner - INFO - [Fold 2] Score=0.9934
2025-05-08 14:12:17,287 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cuda.
2025-05-08 14:12:17,288 - my_library.validations.validation_runner - INFO - [Fold 3] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=True
2025-05-08 14:12:17,289 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-08 14:12:17,289 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200
[W 2025-05-08 14:13:28,254] Trial 1 failed with parameters: {'

KeyboardInterrupt: 